In [14]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
samplesperhost = df_meta['host_id'].value_counts()
samplesperhost

14.1    30
16.2    29
12.2    28
3.2     28
8.2     28
        ..
48.1     8
46.1     7
11.1     7
46.2     6
11.2     6
Name: host_id, Length: 80, dtype: int64